# Тематическое моделирование


Генерируем топ-10 тематических предложений для каждого абзаца

Находим пересечение сетов полученных тематических предложений друг с другом

In [1]:
from gensim.corpora import Dictionary

from gensim.models import LdaMulticore
from gensim.corpora import Dictionary
from gensim import corpora, models

from gensim.models import Phrases

from collections import OrderedDict

import operator

C:\Users\Asus\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
def tokenize_file(file):
    """Получаем на вход файл и отдаем лист листов из токенов"""
    tokenized_text = []
    with open (file, "r", encoding = "utf-8") as file:
        for line in file:
            tokenized_line = line.split(' ')
            tokenized_line = [tok.replace('\n', '')  for tok in tokenized_line if (len(tok) > 2)]
            if(tokenized_line):
                tokenized_text.append(tokenized_line)
    return tokenized_text

In [4]:
tokenized_rzn = tokenize_file("speech_rzn_processed.txt")
tokenized_tmn = tokenize_file("speech_tmn_processed.txt")
tokenized_tula = tokenize_file("speech_tula_processed.txt")
tokenized_yml = tokenize_file("speech_yml_processed.txt")
tokenized_putin = tokenize_file("speech_putin_processed.txt")

In [31]:
def get_topics_tfidf(tokenized_text):
    """производим моделирование 10 тем, возвращаем сет из уникальных слов в созданных темах"""
    dictionary = Dictionary(tokenized_text)
    dictionary.filter_extremes(no_above=0.1)
    bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_rzn]
    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    lda_model_tfidf = LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)     
    topic_words_list = []
    x=lda_model_tfidf.show_topics(num_topics=10,formatted=False)
    topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]
    for topic,words in topics_words:
        topic_words_list += words    
    return set(topic_words_list)
    
get_topics_tfidf(tokenized_tula)#проверяем пример вывода

{'2018',
 'возможность',
 'время',
 'выполнять',
 'город',
 'городской',
 'государственный',
 'гражданин',
 'должный',
 'дорога',
 'жизнь',
 'житель',
 'задача',
 'идти',
 'использовать',
 'каждый',
 'качество',
 'крупный',
 'место',
 'млн',
 'млрд',
 'муниципальный',
 'направлять',
 'начало',
 'начинать',
 'необходимо',
 'областной',
 'образование',
 'общественный',
 'объект',
 'объем',
 'первый',
 'план',
 'планировать',
 'повышение',
 'поддержка',
 'позволять',
 'помощь',
 'правительство',
 'предприятие',
 'президент',
 'проблема',
 'продолжать',
 'развивать',
 'ребенок',
 'результат',
 'решение',
 'россия',
 'рост',
 'самый',
 'свой',
 'система',
 'следующий',
 'создавать',
 'создание',
 'социальный',
 'становиться',
 'строительство',
 'сфера',
 'труд',
 'улучшение',
 'уровень',
 'условие',
 'услуга',
 'федеральный',
 'хотеть',
 'центр',
 'человек',
 'число'}

In [ ]:
#Попарно без повторений сравниваем сеты слов из топ-10 тем каждого текста. Полчаем выводы

In [32]:
text_dict = {'ryazan':tokenized_rzn,"tyumen":tokenized_tmn,"tula": tokenized_tula, 
             "yamal": tokenized_yml, "putin":tokenized_putin}
set_dict = {'ryazan':set(),"tyumen":set(),"tula": set(), "yamal": set(), "putin":set()}
#подсчитываем сеты слов из тем всех данных текстов
for key, value in text_dict.items():
    set_dict[key] = get_topics_tfidf(text_dict[key])
#создаем упорядоченный словарь, чтобы итерироваться по индексам во избежании повторных сравнений
putin_similarity_dict = {'ryazan':0,"tyumen":0,"tula": 0, "yamal": 0}
ordered_set_dict = OrderedDict(set_dict)
keys = list(ordered_set_dict.keys())
values = list(ordered_set_dict.values())
#производим сравнение и выводим результаты
for dict_ind in range(len(keys)):
    current_topic_set = values[dict_ind]
    for compare_ind in range(dict_ind + 1,len(keys)):
        print(keys[dict_ind], "vs", keys[compare_ind])
        compared_topic_set = values[compare_ind]
        inersection = current_topic_set.intersection(compared_topic_set)
        print("Topics intersection is", inersection)
        if(keys[dict_ind] != 'putin'):
            length_initial = len(set_dict[key])
            similarity = len(inersection)/length_initial
            print("intersection size/intial set size= ", similarity)
            putin_similarity_dict[keys[dict_ind]] = similarity
    print( )
print(putin_similarity_dict)
sorted_x = sorted(putin_similarity_dict.items(), key=operator.itemgetter(1))
print(sorted_x[-1][0],"is most similar to Putin speech")

ryazan vs tyumen
Topics intersection is {'оставаться', 'система', 'житель', 'общественный', 'хороший', 'самый', 'результат', 'первый', 'центр'}
intersection size/intial set size=  0.12
ryazan vs tula
Topics intersection is {'труд', 'жизнь', 'общественный', 'место', 'центр', 'областной', 'система', 'житель', 'создавать', 'позволять', 'должный', 'повышение', 'программа', 'результат', 'объект', 'задача', 'качество', 'становиться', 'город', 'продолжать', 'нужно', 'ребенок', 'время', 'предприятие', 'инфраструктура', 'сфера', 'хотеть', 'уровень', 'первый'}
intersection size/intial set size=  0.38666666666666666
ryazan vs yamal
Topics intersection is {'президент', 'направление', 'контроль', 'значимый', 'федеральный', 'инвестиционный', 'хороший', 'объем', 'реализация', 'построить', 'центр', 'организация', 'оставаться', 'рубль', 'являться', 'компания', 'повышение', 'программа', 'участие', 'объект', 'счет', 'задача', 'качество', 'город', 'внимание', 'технология', 'продолжать', 'ребенок', 'время'